In [141]:
import sys
sys.path.append('../Libs')

from libs import check_models

In [142]:
import numpy as np
import pandas as pd
import os


train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

full_data = [train, test]

In [143]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [144]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [145]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [146]:
train[:3]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [147]:
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    dataset.drop(['SibSp', 'Parch'], axis=1, inplace=True)
    
for dataset in full_data:
    dataset['isAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'isAlone'] = 1
    
train[:3]

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,FamilySize,isAlone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,NaN,S,2,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C85,C,2,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,NaN,S,1,1


In [148]:
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

In [149]:
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
    dataset['CategoricalFare'] = pd.qcut(dataset['Fare'], 4)

In [150]:
for dataset in full_data:
    dataset['Age'] = dataset['Age'].fillna(train['Age'].median())
    dataset['CategoricalAge'] = pd.qcut(dataset['Age'], 6)

In [151]:
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'FamilySize', 'Age', 'Fare']

for dataset in full_data:
    dataset.drop(drop_elements, axis=1, inplace=True)
    
train[:3]

,Survived,Pclass,Sex,Embarked,isAlone,CategoricalFare,CategoricalAge
0,0,3,male,S,0,"(-0.001, 7.91]","(19.0, 25.0]"
1,1,1,female,C,0,"(31.0, 512.329]","(31.0, 40.5]"
2,1,3,female,S,1,"(7.91, 14.454]","(25.0, 28.0]"


In [152]:
X = pd.get_dummies(train[train.columns[~train.columns.isin(['Survived'])]])
X_test = pd.get_dummies(test[test.columns[~test.columns.isin(['Survived'])]])
    
y = train['Survived']
X[:3]

,Pclass,isAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,"CategoricalFare_(-0.001, 7.91]","CategoricalFare_(7.91, 14.454]","CategoricalFare_(14.454, 31.0]","CategoricalFare_(31.0, 512.329]","CategoricalAge_(0.419, 19.0]","CategoricalAge_(19.0, 25.0]","CategoricalAge_(25.0, 28.0]","CategoricalAge_(28.0, 31.0]","CategoricalAge_(31.0, 40.5]","CategoricalAge_(40.5, 80.0]"
0,3,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0
1,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0
2,3,1,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0


In [153]:
X_test[:3]

,Pclass,isAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,"CategoricalFare_(-0.001, 7.896]","CategoricalFare_(7.896, 14.454]","CategoricalFare_(14.454, 31.472]","CategoricalFare_(31.472, 512.329]","CategoricalAge_(0.169, 21.0]","CategoricalAge_(21.0, 25.0]","CategoricalAge_(25.0, 28.0]","CategoricalAge_(28.0, 30.0]","CategoricalAge_(30.0, 42.0]","CategoricalAge_(42.0, 76.0]"
0,3,1,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0
1,3,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1
2,2,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1


In [154]:
check_models(X, y)

,Model,Score
0,Support Vector Machines,0.82
1,KNN,0.86
2,Logistic Regression,0.80
3,Random Forest,0.88
4,Naive Bayes,0.76
5,Perceptron,0.76
6,Stochastic Gradient Decent,0.70
7,Linear SVC,0.79
8,Decision Tree,0.88


In [155]:
from sklearn.tree import DecisionTreeClassifier


decision_tree = DecisionTreeClassifier()
decision_tree.fit(X, y)

predict = decision_tree.predict(X_test)

In [157]:
submission = pd.DataFrame({
        'PassengerId': test['PassengerId'],
        'Survived': predict
    })
submission.to_csv('/output/submission.csv', index=False)

KeyError: 'PassengerId'